In [ ]:
from openai import OpenAI, AsyncOpenAI
from langchain.schema import SystemMessage, HumanMessage
from imagepig import ImagePig
from base64 import b64decode
from pathlib import Path
import requests
import datetime
import re
from pydantic import BaseModel
from typing import List, Dict, Tuple
import asyncio
from PIL import Image
from io import BytesIO

API_KEY_PIG = ""
google_key = ""
weatherAPI_key = ""

model_id = "gemini-2.0-flash"
# client = OpenAI(
client = AsyncOpenAI(
        base_url="https://generativelanguage.googleapis.com/v1beta/openai/",
        api_key=google_key
)

In [ ]:
class WeatherInfoAgent:
    @staticmethod
    async def run(location):
        api_key = weatherAPI_key
        url = f"http://api.weatherapi.com/v1/current.json?key={api_key}&q={location}&lang=pt"
        response = requests.get(url)
        return response.json()
    
class ImageGenerationAgent:
    def __init__(self, openai_api_key, image_api_key):
        ImagePig(image_api_key)
        self.image_api_key = image_api_key
        self.openai_api_key = openai_api_key

    async def refine_prompt(self, user_request):
        """Refina o prompt do usuário usando o modelo Gemini 2.0."""
        print("🎨 Refinando prompt...")
        completion = await client.chat.completions.create(
            extra_body={},
            model=model_id,
            messages=[{"role": "user",
                       "content": f"""Transform the following simple description of an image into a detailed prompt for an image generation model. The output should contain only the generated prompt, without any additional explanations or comments. Use rich descriptions, details about colors, textures, lighting, and environment, ensuring that the image is well represented.
                            Example:
                            Input: 'I would like a picture of a pig wearing a shirt.'
                            Output: 'A pink pig with mud on its feet in a pig pen. The pig in question is wearing a textured green shirt. The image is naturally lit from the sky. A farm with details such as a house, crops, and the pig pen itself can be seen around it.' 
                            Now, generate a detailed prompt for the following description: {user_request}"""
                        }])
        refined_prompt = completion.choices[0].message.content
        print(f"✅ Prompt refinado: {refined_prompt}")
        return refined_prompt

    async def generate_image(self, prompt):
        """Gera uma imagem baseada no prompt refinado."""
        print("🖼️ Gerando imagem...")
        response = requests.post(
            "https://api.imagepig.com/",
            headers={"Api-Key": self.image_api_key},
            json={"prompt": prompt},
        )
        if response.ok:
            image_data = b64decode(response.json()["image_data"])
            image_path = Path("generated_image.jpeg")
            with image_path.open("wb") as f:
                f.write(image_data)
            print(f"✅ Imagem gerada e salva em: {image_path}")
            return image_path
        else:
            response.raise_for_status()

    def show_image(self, image_path):
        """Exibe a imagem gerada."""
        with image_path.open("rb") as f:
            img = Image.open(f)
            img.show()

    async def run(self, user_request):
        """Executa o pipeline de geração de imagens."""
        refined_prompt = await self.refine_prompt(user_request)
        image_path = await self.generate_image(refined_prompt)
        self.show_image(image_path)
        return image_path
    
imageGenerationAgent = ImageGenerationAgent(google_key, API_KEY_PIG)
get_weather_info = WeatherInfoAgent()

In [5]:
FINAL_ANSWER_TOKEN = "Final Answer:"
OBSERVATION_TOKEN = "Observation:"
THOUGHT_TOKEN = "Thought:"
PROMPT_TEMPLATE = """Today is {today} and you can use tools to get new information. Answer the question as best as you can using the following tools: 

{tool_description}

Use the following format:

Question: the input question you must answer
Thought: comment on what you want to do next
Action: the action to take, exactly one element of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation repeats N times, use it until you are sure of the answer)
Thought: I now know the final answer
Final Answer: your final answer to the original input question

**Important:** Do not skip steps. Always provide multiple Thoughts before giving the Final Answer.

Begin!

Question: {question}
Thought: {previous_responses}

"""



In [ ]:
async def parse(generated):
    if FINAL_ANSWER_TOKEN in generated:
        return "Final Answer", generated.split(FINAL_ANSWER_TOKEN)[-1].strip()

    regex = r"Action: [\[]?(.*?)[\]]?[\n]*Action Input:[\s]*(.*)"
    match = re.search(regex, generated, re.DOTALL)

    if not match:
        return "Final Answer", generated.strip()

    tool = match.group(1).strip()
    tool_input = match.group(2)
    return tool, tool_input.strip(" ").strip('"')

async def decide_next_action(prompt, request, stop_pattern):
    generated = await client.chat.completions.create(
        model=model_id,
        messages=[
            {"role": "system", "content": prompt},
            {"role": "user", "content": request}
        ],
        stop = stop_pattern
    )
    generated_text = generated.choices[0].message.content  # Extraindo o texto gerado
    tool, tool_input = await parse(generated_text)
    return generated_text, tool, tool_input

async def run_llm(request, show_CoT=False):
    # global messages
    previous_responses = []
    max_loops = 15
    num_loops = 0
    stop_pattern: List[str] = [f'\n{OBSERVATION_TOKEN}', f'\n\t{OBSERVATION_TOKEN}']
    prompt_template: str = PROMPT_TEMPLATE
    tool_description: str = """
        ImageGenerationAgent: Generates images based on detailed descriptions.
        WeatherInfoAgent: Fetches the current weather information for a given location.
    """
    tool_names: str = """
        ImageGenerationAgent, WeatherInfoAgent
    """
    tools = {
        "ImageGenerationAgent": imageGenerationAgent,
        "WeatherInfoAgent": get_weather_info
    }

    prompt = prompt_template.format(
        today               = datetime.date.today(),
        tool_description    = tool_description,
        tool_names          = tool_names,
        question            = request,
        previous_responses = '{previous_responses}'
    )

    print("\n=== BEGIN! ===\n")
    # print(prompt.format(previous_responses=''))
    while num_loops < max_loops:
        num_loops += 1
        curr_prompt = prompt.format(previous_responses='\n'.join(previous_responses))
        generated, tool, tool_input = await decide_next_action(curr_prompt, request, stop_pattern)
        if show_CoT:  
            print(f"\n=== Loop Number {num_loops} ===\n", flush=True)
            print(generated, flush=True)

        if tool == 'Final Answer':
            if show_CoT:
                print("\n=== Final Answer ===\n", flush=True)
                print(tool_input, flush=True)
            return tool_input
        if tool not in tool_names:
            raise ValueError(f"Unknown tool: {tool}")
        
        tool_result = await tools[tool].run(tool_input)
        
        # generated += f"\n{OBSERVATION_TOKEN} {tool_result}\n{THOUGHT_TOKEN}"
        response_text = f"\n{OBSERVATION_TOKEN} {tool_result}\n{THOUGHT_TOKEN}"
        generated += response_text

        if show_CoT:
            print(response_text, flush=True)

        previous_responses.append(generated)

In [13]:
await run_llm("Eu comprei 6 maças, em seguida fui na roça e colhi mais 5 maças do pé e comi 2 enquanto fazia isso."
    " Depois de colher, entreguei uma a minha irmã e uma a minha mãe. Em seguida utilizei 3 para fazer uma torta. "
    "Quantas maças me sobraram?",show_CoT=True
)


=== INICIANDO O PROCESSO ===


=== Loop Number 1 ===

Thought: Okay, let's break down the problem step by step to determine how many apples are left.

1. **Start:** You begin with 6 apples.
2. **Picking apples:** You pick 5 more apples. Total: 6 + 5 = 11 apples.
3. **Eating apples:** You eat 2 apples. Total: 11 - 2 = 9 apples.
4. **Giving apples:** You give 1 apple to your sister and 1 to your mother. Total: 9 - 1 - 1 = 7 apples.
5. **Making a pie:** You use 3 apples for a pie. Total: 7 - 3 = 4 apples.

Therefore, you have 4 apples remaining.

Final Answer: 4


=== Final Answer ===

4


'4'

In [12]:
await run_llm("Eu comprei 6 maças, em seguida fui na roça e colhi mais 5 maças do pé e comi 2 enquanto fazia isso. "
    "Depois de colher, entreguei uma a minha irmã e uma a minha mãe. Em seguida utilizei 3 para fazer uma torta. "
    "Quantas maças me sobraram?",show_CoT=False
)


=== INICIANDO O PROCESSO ===



'Você sobrou com 4 maças.'

In [16]:
await run_llm("Eu comprei 6 maças, em seguida fui na roça e colhi mais 5 maças do pé e comi 2 enquanto fazia isso. "
    "Depois de colher, entreguei uma a minha irmã e uma a minha mãe. Em seguida utilizei 3 para fazer uma torta. "
    "Quantas maças me sobraram? Faça uma imagem do número de maças da resposta final",show_CoT=True
)


=== INICIANDO O PROCESSO ===


=== Loop Number 1 ===

Thought: Primeiro, preciso calcular o número total de maçãs que a pessoa tinha. Inicialmente, ela tinha 6 maçãs compradas. Depois, colheu mais 5. Então, o total de maçãs é 6 + 5 = 11.

Em seguida, preciso subtrair as maçãs que ela comeu, deu para a irmã, para a mãe e usou na torta. Ela comeu 2, deu 1 para a irmã e 1 para a mãe, e usou 3 na torta. Então, o total de maçãs que saíram é 2 + 1 + 1 + 3 = 7.

Finalmente, subtraio as maçãs que saíram do número total de maçãs: 11 - 7 = 4.

Agora, gero uma imagem com 4 maçãs.
Action: ImageGenerationAgent
Action Input: Four red apples arranged in a visually appealing way.
🎨 Refinando prompt...
✅ Prompt refinado: Four vibrant, crimson red apples with subtle variations in shape and size, arranged artfully on a dark, polished wooden table. Light reflects off their glossy skins, highlighting small imperfections and subtle color variations, from deep scarlet to hints of orange. One apple has a sin

HTTPError: 429 Client Error: Too Many Requests for url: https://api.imagepig.com/

In [15]:
await run_llm("Qual a temperatura em Minas Gerais hoje?",show_CoT=True
)


=== INICIANDO O PROCESSO ===


=== Loop Number 1 ===

Thought: Para responder a essa pergunta, preciso saber a localização específica dentro de Minas Gerais, pois a temperatura pode variar bastante dependendo da cidade ou região. Em seguida, usarei a ferramenta de busca do clima para obter a temperatura.

Action: WeatherInfoAgent
Action Input: Belo Horizonte, Minas Gerais

Observation: {'location': {'name': 'Belo Horizonte', 'region': 'Minas Gerais', 'country': 'Brazil', 'lat': -19.9167, 'lon': -43.9333, 'tz_id': 'America/Sao_Paulo', 'localtime_epoch': 1741548485, 'localtime': '2025-03-09 16:28'}, 'current': {'last_updated_epoch': 1741547700, 'last_updated': '2025-03-09 16:15', 'temp_c': 31.1, 'temp_f': 88.0, 'is_day': 1, 'condition': {'text': 'Parcialmente nublado', 'icon': '//cdn.weatherapi.com/weather/64x64/day/116.png', 'code': 1003}, 'wind_mph': 4.3, 'wind_kph': 6.8, 'wind_degree': 270, 'wind_dir': 'W', 'pressure_mb': 1015.0, 'pressure_in': 29.97, 'precip_mm': 0.0, 'precip_in': 0

'A temperatura em Belo Horizonte, Minas Gerais, é de 31.1 graus Celsius.'